In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import pandas as pd

In [3]:
import lib
import plotting

In [4]:
from configparser import ConfigParser
cfg = ConfigParser()
cfg.read("config.cfg")

['config.cfg']

In [5]:
from tqdm._tqdm_notebook import tqdm_notebook as tqdm # we manually import the notebook submodule as the normal one struggles with jupyterlab
tqdm.pandas() # this enables us to use progress_apply instead of apply

In [12]:
results_prefix = cfg.get("results", "prefix")

sequence_file = os.path.join(cfg.get("directory", "exchange"), results_prefix + "001_clustering.p")
labels_file = os.path.join(cfg.get("directory", "exchange"), "[wikidata]004_labels_readable.p")
results_prefix

'[WIKIDATA_10]'

In [7]:
df = pd.read_pickle(sequence_file)
df.head()

,bot_name,bot_sequence,length,length_nobreak,sequence,sequence_readable,user_id,user_name,stat_dist,kmeans_4,kmeans_8
0,False,False,1517,1385,"[REVERT_MANUAL, BREAK, CLAIM_CREATE, BREAK, CL...","[Revert Item, Break, Create Claim, Break, Crea...",1,Hoo man,"[0.033592832488441825, 0.029154644609674315, 0...",0,3
1,False,False,44,23,"[SITELINK_ADD, BREAK, SITELINK_ADD, BREAK, SIT...","[Add Sitelink, Break, Add Sitelink, Break, Add...",100012,Kane 14,"[0.02586206896551723, 0.025862068965517227, 0....",2,1
2,False,False,17,12,"[LABEL_UPDATE, ALIAS_ADD, BREAK, CLAIM_CREATE,...","[Edit Label, Add Item Alias, Break, Create Cla...",1000197,Zaizone,"[0.026205469193709592, 0.07779210211701826, 0....",1,5
3,False,False,91,82,"[SITELINK_UPDATE, BREAK, LABEL_ADD, CLAIM_CREA...","[Edit Sitelink, Break, Add Label, Create Claim...",1001159,Gogo hr,"[0.04118566424353361, 0.03162035177772598, 0.0...",0,0
4,False,False,21,12,"[DESCRIPTION_ADD, BREAK, DESCRIPTION_UPDATE, D...","[Add Description, Break, Edit Description, Add...",1001194,Phylosofo,"[0.08620689655172406, 0.025862068965517255, 0....",1,7


In [8]:
labels = pd.read_pickle(labels_file)['label'].values

In [23]:
num_clusters = 8
df_selector = "kmeans_{n}".format(n=num_clusters)

In [24]:
cluster_group = df.groupby(df_selector)

In [25]:
results = []
for i, g in cluster_group:
    actions_break = g["length"]
    actions = g["length_nobreak"]
    res = {
        "cluster": i,
        "population": len(g),
        "actions_sum": actions.sum(),
        "actions_mean": actions.mean(),
        "actions_median": actions.median(),
        "actions_max": actions.max(),
        "actions_min": actions.min(),
        "num_breaks": actions_break.sum() - actions.sum()
    }
    
    #for i, r in g.iterrows():
    #    if len(r['sequence']) == actions.max():
    #        print(r["user_id"])
    results.append(res)
    
df_res = pd.DataFrame(results)
df_res

,actions_max,actions_mean,actions_median,actions_min,actions_sum,cluster,num_breaks,population
0,1078290,5500.620969,42.0,10,33944332,0,595116,6171
1,15542,177.263363,39.0,10,1591825,1,894356,8980
2,1574515,4146.505503,24.0,10,11303374,2,226571,2726
3,1098458,2908.146329,51.0,10,17350001,3,621802,5966
4,16405,154.283611,28.0,10,639197,4,225239,4143
5,58090,336.568340,42.0,10,2272846,5,543182,6753
6,86583,1048.192671,52.0,10,1773542,6,267967,1692
7,859179,2011.364865,23.0,10,4018707,7,75252,1998


In [26]:
df_res["population"].sum()

38429